# Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from art.utils import load_mnist
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.extraction import KnockoffNets

# Load Data

In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

# Target Model

In [3]:
target_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation="relu"),
    Dense(10)
])

In [4]:
target_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [5]:
target_model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=1)

Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2203 - accuracy: 0.9335
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1280 - accuracy: 0.9621
Epoch 3/5
938/938 [==============================] - 4s 5ms/step - loss: 0.1165 - accuracy: 0.9666
Epoch 4/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1038 - accuracy: 0.9699
Epoch 5/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0909 - accuracy: 0.9748


In [6]:
target_classifier = TensorFlowV2Classifier(
    model=target_model, 
    loss_object=CategoricalCrossentropy(from_logits=True), 
    optimizer=Adam(learning_rate=0.01), 
    nb_classes=10,
    input_shape=(28, 28),
    clip_values=(min_, max_)
)

In [7]:
print("Target Model Accuracy:", target_model.evaluate(x_test, y_test, verbose=0)[1])

Target Model Accuracy: 0.9574999809265137


# CopycatCNN

In [8]:
knockoff_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation="relu"),
    Dense(10)
])

In [9]:
knockoff_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [10]:
knockoff_classifier = TensorFlowV2Classifier(
    model=knockoff_model, 
    loss_object=CategoricalCrossentropy(from_logits=True), 
    optimizer=Adam(learning_rate=0.01),
    nb_classes=10,
    input_shape=(28, 28),
    clip_values=(min_, max_)
)

In [11]:
knockoff_attack = KnockoffNets(
    classifier=knockoff_classifier, 
    nb_epochs=10,
    nb_stolen=10000
)

In [13]:
knockoff_attack.extract(x_test, y_test, thieved_classifier=target_classifier)

art.estimators.classification.tensorflow.TensorFlowV2Classifier(model=<keras.src.engine.sequential.Sequential object at 0x7fa41afe85b0>, nb_classes=10, input_shape=(28, 28), loss_object=<keras.src.losses.CategoricalCrossentropy object at 0x7fa3f80be1d0>, optimizer=<keras.src.optimizers.adam.Adam object at 0x7fa3f80be200>, train_step=None, channels_first=False, clip_values=array([0., 1.], dtype=float32), preprocessing_defences=None, postprocessing_defences=None, preprocessing=StandardisationMeanStdTensorFlow(mean=0.0, std=1.0, apply_fit=True, apply_predict=True))

In [14]:
print("CopycatCNN Model Accuracy:", knockoff_classifier._model.evaluate(x_test, y_test, verbose=0)[1])

CopycatCNN Model Accuracy: 0.09610000252723694
